In [1]:
import os
from docx import Document
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import fitz
import json
import hashlib
import shutil

/Users/sharathsolomon/Desktop/SSAA Chatbot/Development/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load embedding model
# model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# model.save('models/my_miniLM_model')

In [2]:
model = SentenceTransformer('models/my_miniLM_model')

# Data Preprocessing

In [3]:

path = os.path.join(os.getcwd(),'SSAA Documents/Valid Documents')
files = os.listdir(path)
files

['0685i00000GMzwSAAT.pdf',
 '068Mo00000OA0dhIAD.pdf',
 '0685i00000COnTYAA1.pdf',
 '0685i00000CPMiuAAH.pdf',
 '0685i00000JTytPAAT.pdf',
 '068Mo00000TExCQIA1.pptx',
 '0685i00000CRB13AAH.docx',
 'Final_Member Webinar_Storage Agreements 2023.pdf',
 '0685i00000CPMvQAAX.pdf',
 '0685i00000CQGYpAAP.docx',
 'SSAA-Ski-Summit-Summary.pdf',
 '0685i00000COmvjAAD.pdf',
 '0685i00000CPZoMAAX.pdf',
 '0685i00000CQfvpAAD.pdf',
 '0685i00000CQ2jRAAT.pdf',
 '068J3000002BYLxIAO.docx',
 '0685i00000MY8fWAAT.pdf',
 '068Mo00000Oxq9iIAB.pdf',
 '068Mo00000OxXXpIAN.pdf',
 '0685i00000COnSkAAL.pdf',
 '0685i00000CQ39TAAT.pdf',
 'SSAA Arrears Management Guide 2024 (2).pdf',
 '0685i00000CQKadAAH.docx',
 '0685i00000CQFYEAA5.pdf',
 '0685i00000Hcg2QAAR.pdf',
 '0685i00000HchNzAAJ.pdf',
 'Urbis-Storage-Index-December-2020-8p.pdf',
 '068J3000003eZdcIAE.pdf',
 '068J3000002BzyhIAC.docx',
 '0685i00000CPMZqAAP.pptx',
 '0685i00000CQ2KDAA1.docx',
 '0685i00000CQ2i2AAD.pdf',
 '0685i00000HdnQDAAZ.pdf',
 '068Mo00000PmdK7IAJ.pdf',
 '068

In [4]:
# Step 1: Load .docx and extract paragraphs
def load_docx_text(path):
    doc = Document(path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    return paragraphs

In [ ]:
# # Step 2: Chunk paragraphs into simulated pages
# def chunk_into_pages(paragraphs, max_words=800):
#     pages = []
#     current_page = []
#     word_count = 0

#     for para in paragraphs:
#         words = para.split()
#         if word_count + len(words) > max_words:
#             pages.append(' '.join(current_page))
#             current_page = []
#             word_count = 0
#         current_page.append(para)
#         word_count += len(words)
    
#     if current_page:
#         pages.append(' '.join(current_page))
#     return pages

In [19]:
def chunk_into_pages(paragraphs, max_length=800):
    pages = []
    current = ""
    for para in paragraphs:
        if len(current) + len(para) < max_length:
            current += para + " "
        else:
            pages.append(current.strip())
            current = para + " "
    if current:
        pages.append(current.strip())
    return pages

In [20]:
def extract_docx(file):
    path = os.path.join(os.getcwd(),f'SSAA_Documents/Valid Documents/{file}')
    paragraphs = load_docx_text(path)
    pages = chunk_into_pages(paragraphs)
    return pages    

In [ ]:
# def extract_pdf(file):
#     path = os.path.join(os.getcwd(),f'Documents/{file}')
#     doc = fitz.open(path)
#     pages = []
#     for page in doc:
#         text = page.get_text().strip()
#         if text:
#             pages.append(text)
    
#     doc.close()
#     return pages

In [21]:
def extract_pdf(file):
    path = os.path.join(os.getcwd(), f'SSAA_Documents/Valid Documents/{file}')
    doc = fitz.open(path)
    pages = [page.get_text().strip() for page in doc if page.get_text().strip()]
    doc.close()
    return pages

In [8]:
def hash_document_text(text):
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

In [9]:
def extract_full_text(file_path):
    ext = file_path.split('.')[-1].lower()
    file_name = os.path.basename(file_path)
    if ext == 'pdf':
        return "\n".join(extract_pdf(file_name))
    elif ext == 'docx':
        return "\n".join(extract_docx(file_name))
    else:
        return None

In [22]:
def remove_duplicate_documents(doc_folder='SSAA_Documents/Valid Documents'):
    seen_hashes = set()
    doc_path = os.path.join(os.getcwd(), doc_folder)
    files = os.listdir(doc_path)

    # Optional: move duplicates to separate folder
    duplicates_folder = os.path.join(os.getcwd(), "Duplicate_Documents")
    os.makedirs(duplicates_folder, exist_ok=True)

    for file in files:
        full_path = os.path.join(doc_path, file)

        if not file.endswith(('.pdf', '.docx')):
            continue

        text = extract_full_text(full_path)
        if not text:
            continue

        doc_hash = hash_document_text(text.strip())

        if doc_hash in seen_hashes:
            # Duplicate found: move to duplicates folder
            shutil.move(full_path, os.path.join(duplicates_folder, file))
            print(f"❌ Removed duplicate: {file}")
        else:
            seen_hashes.add(doc_hash)

    print("✅ Duplicate removal complete.")

In [23]:
remove_duplicate_documents()

✅ Duplicate removal complete.


In [24]:
files = os.listdir('SSAA_Documents/Valid Documents')
len(files)

184

In [25]:
def preprocessing():
    all_pages = []
    metadata = []
    seen_hashes = set()

    path = os.path.join(os.getcwd(), 'SSAA_Documents/Valid Documents')
    files = os.listdir(path)

    for file in files:
        if file.split('.')[-1] == 'docx':
            pages = extract_docx(file)
        elif file.split('.')[-1] == 'pdf':
            pages = extract_pdf(file)
        else:
            continue

        for i, page in enumerate(pages):
            content_hash = hash_document_text(page.strip())

            if content_hash in seen_hashes:
                continue  # Skip duplicate content

            seen_hashes.add(content_hash)
            all_pages.append(page)
            metadata.append({
                "document": file,
                "page_number": i + 1,
                "text": page
            })

    return all_pages, metadata

In [ ]:
# def preprocessing():
#     all_pages = []
#     metadata = []
#     path = os.path.join(os.getcwd(),'Documents')
#     files = os.listdir(path)

#     for file in files:
#         if file.split('.')[-1] in ['docx']:
#             pages = extract_docx(file)
            
#         elif file.split('.')[-1] in ['pdf']:
#             pages = extract_pdf(file)
#         else:
#             continue

#         for i, page in enumerate(pages):
#                 all_pages.append(page)
#                 metadata.append({
#                     "document": file,
#                     "page_number": i + 1,
#                     "text": page
#                 })
#     return (all_pages,metadata)

In [26]:
all_pages, metadata = preprocessing()

In [27]:
len(metadata)

2513

# Data Indexer

In [28]:
def vectorize_pages(pages,model):
    embeddings = model.encode(pages, convert_to_numpy=True)
    return embeddings

In [29]:
def indexer_faiss():
    embeddings = vectorize_pages(all_pages,model)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)


    # 1. First time: after indexing
    faiss.write_index(index, "vector_index.faiss")
    with open("metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata, f)


In [30]:
indexer_faiss()

In [31]:
def search_index(query, index, metadata_list, model, top_k=3):
    query_vec = model.encode([query])
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        metadata = metadata_list[idx]
        results.append({
            "document": metadata["document"],
            "page_number": metadata["page_number"],
            "text": metadata["text"][:500],  # preview
            "score": D[0][list(I[0]).index(idx)]
        })
    return results

In [32]:

# Loading Indexer and metadata
index = faiss.read_index("vector_index.faiss")
with open("metadata.json", "r") as f:
    metadata_list = json.load(f)

# Example query
query = "What is the purpose of an Alternate Contact Person (ACP) and what rights do they have?"
results = search_index(query, index, metadata_list, model)

print(f'Query : {query}')
# Display results
for res in results:
    print(f"\n📄 Document: {res['document']} | Page: {res['page_number']} | Score: {res['score']:.2f}")
    print(f"📝 Content Preview:\n{res['text']}...")
    print('-' * 80)

Query : What is the purpose of an Alternate Contact Person (ACP) and what rights do they have?

📄 Document: 068Mo00000Oxz9hIAB.pdf | Page: 10 | Score: 0.63
📝 Content Preview:
Access to a space by others – 
alternate contact person
• ACP is an agent whose decisions in relation to the 
agreement are binding on the storer.
• Can be contacted in case the storer cannot be 
reached. Can discuss any matter relating to the 
agreement with them.
ACP can access the Space to remove the goods when 
required under the agreement (for eg after 
termination), if the storer is absent, unwilling or unable 
to remove the goods....
--------------------------------------------------------------------------------

📄 Document: 0685i00000CQRmRAAX.docx | Page: 10 | Score: 0.77
📝 Content Preview:
We may disclose, sell or transfer to third parties any non-personal, aggregated information that we collect from you and our other customers. alternate contact person It is your responsibility to ensure that you have o

In [66]:
results

[{'document': '0685i00000GKKY5AAP.docx',
  'page_number': 4,
  'text': 'Alternative Contact Person or ACP means the alternative contact person that You specify in the Schedule. Associate means the ACP, Your agent and/or a third party: (a) who enters the Space (or the Facility) at Your request, invitation or direction; or (b) whose entry to the Space (or the Facility) was facilitated by any of Your acts or omissions, including, but not limited to, the provision of a key, access card, codes or Bluetooth-enabled device; or (c) who gains unauthorised entry to the Space ',
  'score': np.float32(0.5995209)},
 {'document': '068Mo00000Oxz9hIAB.pdf',
  'page_number': 10,
  'text': 'Access to a space by others – \nalternate contact person\n• ACP is an agent whose decisions in relation to the \nagreement are binding on the storer.\n• Can be contacted in case the storer cannot be \nreached. Can discuss any matter relating to the \nagreement with them.\nACP can access the Space to remove the goods 

In [5]:
import openai
from dotenv import load_dotenv
load_dotenv()
# Set your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI(api_key=api_key)  # Replace with your actual key

def get_answer_from_openai(query: str, content: str, model: str = "gpt-3.5-turbo-0125") -> str:
    """
    Uses OpenAI's ChatCompletion API to answer a question based on the provided context.

    Args:
        query (str): The user's question.
        content (str): The context/content from which the answer should be extracted.
        model (str): OpenAI model to use (default is gpt-3.5-turbo-0125).

    Returns:
        str: The generated answer from OpenAI.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the given context."},
                {"role": "user", "content": f"Context: {content}\n\nQuestion: {query}"}
            ],
            temperature=0.2,
            max_tokens=300
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error occurred: {e}"

In [7]:
def generate_answer(query):
    results = search_index(query, index, metadata_list, model)
    context=''
    for res in results:
        context += res['text']
    
    answer = get_answer_from_openai(query,context)
    return answer

In [8]:
query = "deceased storer"
generate_answer(query)

'When a Storer passes away, the Facility can only release the stored goods to the person who has been granted probate by the court. If the Storer dies without a will, it is referred to as "intestate." In such cases, the next of kin will need to provide proof of being the executor or administrator of the deceased\'s estate in order to access the stored items. The executor or administrator should sign the necessary authorization forms to gain access to the storage unit and its contents.'

In [ ]:
# # Example query
# query = "deceased storer"
# results = search_index(query, index, metadata_list, model)
# context=''
# for res in results:
#     context += res['text']

# print(f'Query : {query}')
# # Display results
# for res in results:
#     print(f"\n📄 Document: {res['document']} | Page: {res['page_number']} | Score: {res['score']:.2f}")
#     print(f"📝 Content Preview:\n{res['text']}...")
#     print('-' * 80)

# answer = get_answer_from_openai(query,context)
# print(f'Answer from OpenAI model :\n{answer}')

Query : deceased storer

📄 Document: NZ-Customer-Storage-Agreement-Guidelines.v1.04-January-2023.pdf | Page: 47 | Score: 0.91
📝 Content Preview:
SSAA CUSTOMER STORAGE AGREEMENT GUIDELINES (CSA) 
© Self Storage Association of Australasia 2023 
47 
KJW - 449872/6 - 194743.3 
 
 
 
19. 
DECEASED STORER 
 
 
When a Storer dies, a Facility is not able to ‘release’ goods to any person other than the person to 
whom probate is granted. A probate document is an official court issued document. It will include a 
court stamp. 
 
19.1 Storer dies without a will 
Where a person dies without a will it is called “intestate”. The next of kin will need ...
--------------------------------------------------------------------------------

📄 Document: Self Storage Legal Landscape - Australia_Ritika Sardar and Ethan Holden.pdf | Page: 12 | Score: 0.95
📝 Content Preview:
Access to a space by others – relatives/executor 
when storer deceased (illustration cont.)
• On a storer’s death, the lock should not be